In [2]:
import numpy as np

def v_cycle(f, u, N, gamma=1):
    """
    V-cycle multigrid algorithm for solving the 1D Poisson equation
    u''(x) = f(x) on [0, 1] with Dirichlet boundary conditions
    """
    # Define the coarse grid
    h = 1/(N+1)
    x_coarse = np.linspace(h, 1-h, N)
    f_coarse = f(x_coarse)
    u_coarse = np.zeros(N)
    
    # Pre-smooth on the fine grid
    u = relax(f, u, gamma)
    
    # If we're on the coarsest grid, solve exactly and return
    if N == 1:
        u_coarse[0] = f_coarse[0]/2
        return u_coarse
    
    # Restrict the residual to the coarse grid
    r = f - laplacian(u)/h**2
    r_coarse = restrict(r, N)
    
    # Solve the coarse problem recursively
    e_coarse = v_cycle(r_coarse, u_coarse, N//2, gamma)
    
    # Interpolate the error to the fine grid
    e_fine = interpolate(e_coarse, N)
    u += e_fine
    
    # Post-smooth on the fine grid
    u = relax(f, u, gamma)
    
    return u

def laplacian(u):
    # Compute the second derivative of u using a central difference scheme
    h = 1/(len(u)+1)
    uxx = np.zeros_like(u)
    for i in range(1, len(u)-1):
        uxx[i] = (u[i-1] - 2*u[i] + u[i+1])/h**2
    return uxx

def restrict(r, N):
    # Restrict the residual from the fine grid to the coarse grid
    r_coarse = np.zeros(N//2)
    for i in range(N//2):
        r_coarse[i] = r[2*i+1] - 2*r[2*i] + r[2*i-1]
    return r_coarse

def interpolate(e_coarse, N):
    # Interpolate the error from the coarse grid to the fine grid
    e_fine = np.zeros(N)
    for i in range(N//2):
        e_fine[2*i+1] += e_coarse[i]/2
        e_fine[2*i] += e_coarse[i]/2
    return e_fine

def relax(f, u, gamma):
    # Perform gamma iterations of Gauss-Seidel relaxation on the equation
    # u''(x) = f(x) with Dirichlet boundary conditions
    h = 1/(len(u)+1)
    for i in range(gamma):
        for j in range(1, len(u)-1):
            u[j] = 0.5*(h**2*f[j] + u[j-1] + u[j+1])
    return u
